<a href="https://colab.research.google.com/github/SamuelLiuCh/Options/blob/main/%E6%B3%A2%E5%8B%95%E7%8E%87%E9%A0%90%E6%B8%AC%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part 0: Import 套件

In [12]:
#下載資料套件
import urllib3
import requests
import json
from bs4 import BeautifulSoup

#資料處理套件
import math
from math import exp, log, sqrt
import numpy as np
import scipy
from scipy import stats
import pandas as pd
from datetime import datetime, date

#畫圖套件
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.subplots import make_subplots
import plotly.graph_objects as go

#Part 1: 下載資料

In [13]:
#請求網站（選擇權）
http = urllib3.PoolManager()
url = "https://www.taifex.com.tw/cht/3/dlOptDataDown"
res = http.request(
     'POST',
      url,
      fields={
         'down_type': 1,
         'commodity_id': 'TXO',
         'queryStartDate': '2023/02/01',
         'queryEndDate': '2023/02/08'
      }
 )
html_doc = res.data

#使用BeautifulSoup解析資料
soup = BeautifulSoup(html_doc, 'html.parser')
soup_str = str(soup)
lines = soup_str.split('\r\n')

# 新增空的dataframe,定義欄位名稱
df = pd.DataFrame(columns = lines[0].split(','))

# 把選擇權資料一行一行寫入dataframe內
for i in range(1, len(lines) - 1):
    list_ = lines[i].split(',')[:-1]
    df_length = len(df)
    df.loc[df_length] = list_

/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((


In [14]:
df.head()

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段,漲跌價,漲跌%
0,2023/02/01,TXO,202302W1,13400.0000,買權,-,-,-,-,0,0,0,-,-,-,-,,一般,-,-
1,2023/02/01,TXO,202302W1,13400.0000,買權,-,-,-,-,0,-,-,-,-,-,-,,盤後,-,-
2,2023/02/01,TXO,202302W1,13400.0000,賣權,0.6,0.6,0.6,0.6,1,0,1,-,0.4,0.6,0.6,,一般,0.5,-
3,2023/02/01,TXO,202302W1,13400.0000,賣權,-,-,-,-,0,-,-,-,-,-,-,,盤後,-,-
4,2023/02/01,TXO,202302W1,13500.0000,買權,-,-,-,-,0,0,0,-,-,-,-,,一般,-,-


In [15]:
#請求網站（證券指數）
month_list = pd.date_range('2023-01-01','2023-02-01', freq='MS').strftime("%Y%m%d").tolist()
ms_df = pd.DataFrame()
for month in month_list:
    url_sto = "https://www.twse.com.tw/indicesReport/MI_5MINS_HIST?response=json&date=" + month
    res_sto = requests.get(url_sto)
    stock_json = res_sto.json()
    #stock_json['data']
    pd.DataFrame.from_dict(stock_json['data'])
    stock_df = pd.DataFrame.from_dict(stock_json['data'])
    ms_df = ms_df.append(stock_df, ignore_index = True)
ms_df.columns = stock_json['fields']

In [ ]:
ms_df.head()

In [18]:
for col in range(5):
    for row in range(ms_df.shape[0]):
        # 把"日期"從字串(string)換成時間(datetime)
        if col == 0:
            day = ms_df.iloc[row,0].split('/')
            ms_df.iloc[row, 0] = datetime(int(day[0])+1911, int(day[1]), int(day[2]))  
        # 把字串(string)換成浮點數(float): "開盤價", "最高價", "最低價", "收盤價"並四捨五入到百位數 
        else:
            index = ms_df.iloc[row,col].split(',')
            indexint = index[1].split('.')
            ms_df.iloc[row, col] = int(index[0])*1000+int(indexint[0])
            ms_df.iloc[row, col] = round(ms_df.iloc[row,col]/100)*100

In [ ]:
ms_df.head()

#Part 2: 資料處理

In [20]:
# 資料轉型
for col in [0, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]:
    for row in range(df.shape[0]):
        # 把"日期"從字串(string)換成時間(datetime)
        if col == 0:
            day = df.iloc[row,0].split('/')
            df.iloc[row, 0] = datetime(int(day[0]), int(day[1]), int(day[2]))  
        # 把字串(string)換成浮點數(float): "履約價", "開盤價", "最高價", "最低價", "收盤價", "成交量", "結算價", "未沖銷契約數", "最後最佳買價", "最後最佳賣價", "歷史最高價", "歷史最低價" 
        elif col != 0 and df.iloc[row, col] != '-':
            df.iloc[row, col] = float(df.iloc[row,col])

In [ ]:
df.head()

In [ ]:
df.describe()

In [23]:
df['到期月份(週別)'].unique()

array(['202302W1', '202302W2', '202302  ', '202303  ', '202304  ',
       '202306  ', '202309  ', '202302W4'], dtype=object)

In [24]:
call_df = df.loc[(df['到期月份(週別)'] == '202302W2') & \
                (df['買賣權'] == '買權') & \
                (df['交易時段'] == '一般')]

call_df

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段,漲跌價,漲跌%
212,2023-02-01 00:00:00,TXO,202302W2,13400.0,買權,1890.0,1890.0,1890.0,1890.0,1.0,1990.0,1.0,1970.0,2010.0,1890.0,1890.0,,一般,10,0.53%
216,2023-02-01 00:00:00,TXO,202302W2,13500.0,買權,-,-,-,-,0.0,1890.0,0.0,-,-,-,-,,一般,-,-
220,2023-02-01 00:00:00,TXO,202302W2,13600.0,買權,-,-,-,-,0.0,1790.0,0.0,-,-,-,-,,一般,-,-
224,2023-02-01 00:00:00,TXO,202302W2,13700.0,買權,-,-,-,-,0.0,1690.0,0.0,-,-,-,-,,一般,-,-
228,2023-02-01 00:00:00,TXO,202302W2,13800.0,買權,-,-,-,-,0.0,1590.0,0.0,-,-,-,-,,一般,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7178,2023-02-08 00:00:00,TXO,202302W2,16800.0,買權,-,-,-,-,0.0,0.0,0.0,-,-,-,-,,一般,-,-
7182,2023-02-08 00:00:00,TXO,202302W2,16900.0,買權,-,-,-,-,0.0,0.0,0.0,-,-,-,-,,一般,-,-
7186,2023-02-08 00:00:00,TXO,202302W2,17000.0,買權,-,-,-,-,0.0,0.0,1.0,-,-,-,-,,一般,-,-
7190,2023-02-08 00:00:00,TXO,202302W2,17100.0,買權,-,-,-,-,0.0,0.0,0.0,-,-,-,-,,一般,-,-


In [25]:
# 取得 dataframe index
index = call_df['履約價'].unique()
idx = np.sort(index)
idx

array([13400.0, 13500.0, 13600.0, 13700.0, 13800.0, 13900.0, 14000.0,
       14100.0, 14200.0, 14300.0, 14400.0, 14500.0, 14550.0, 14600.0,
       14650.0, 14700.0, 14750.0, 14800.0, 14850.0, 14900.0, 14950.0,
       15000.0, 15050.0, 15100.0, 15150.0, 15200.0, 15250.0, 15300.0,
       15350.0, 15400.0, 15450.0, 15500.0, 15550.0, 15600.0, 15650.0,
       15700.0, 15750.0, 15800.0, 15850.0, 15900.0, 15950.0, 16000.0,
       16050.0, 16100.0, 16200.0, 16300.0, 16400.0, 16500.0, 16600.0,
       16700.0, 16800.0, 16900.0, 17000.0, 17100.0, 17200.0], dtype=object)

In [26]:
# 取得 dataframe 欄位名稱
dates = call_df['交易日期'].unique()
dates

array([datetime.datetime(2023, 2, 1, 0, 0),
       datetime.datetime(2023, 2, 2, 0, 0),
       datetime.datetime(2023, 2, 3, 0, 0),
       datetime.datetime(2023, 2, 6, 0, 0),
       datetime.datetime(2023, 2, 7, 0, 0),
       datetime.datetime(2023, 2, 8, 0, 0)], dtype=object)

In [27]:
# 建立 dataframe 包含 index 與欄位名稱
call_t_df = pd.DataFrame(index = idx, columns = list(dates))
# 取出各日期的未沖銷契約數 依序放入 dataframe 中
for col in call_t_df.columns:
    call_t_df[col] = call_df.loc[call_df['交易日期'] == col].set_index('履約價')['未沖銷契約數']
#空值填入 0
call_t_df = call_t_df.fillna(0)
call_t_df

,2023-02-01,2023-02-02,2023-02-03,2023-02-06,2023-02-07,2023-02-08
13400.0,1.0,1.0,1.0,1.0,1.0,1.0
13500.0,0.0,0.0,0.0,0.0,0.0,0.0
13600.0,0.0,0.0,0.0,0.0,0.0,0.0
13700.0,0.0,0.0,0.0,0.0,0.0,0.0
13800.0,0.0,0.0,0.0,0.0,0.0,0.0
13900.0,1.0,1.0,1.0,1.0,1.0,1.0
14000.0,0.0,0.0,0.0,0.0,0.0,0.0
14100.0,0.0,0.0,0.0,0.0,0.0,0.0
14200.0,0.0,0.0,0.0,0.0,0.0,0.0
14300.0,1.0,1.0,1.0,1.0,1.0,1.0


In [28]:
put_df = df.loc[(df['到期月份(週別)'] == '202302W2') & \
                (df['買賣權'] == '賣權') & \
                (df['交易時段'] == '一般')]

put_df

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段,漲跌價,漲跌%
214,2023-02-01 00:00:00,TXO,202302W2,13400.0,賣權,-,-,-,-,0.0,0.1,140.0,-,-,-,-,,一般,-,-
218,2023-02-01 00:00:00,TXO,202302W2,13500.0,賣權,-,-,-,-,0.0,0.1,5.0,-,-,-,-,,一般,-,-
222,2023-02-01 00:00:00,TXO,202302W2,13600.0,賣權,-,-,-,-,0.0,0.1,0.0,-,-,-,-,,一般,-,-
226,2023-02-01 00:00:00,TXO,202302W2,13700.0,賣權,-,-,-,-,0.0,0.1,11.0,-,-,-,-,,一般,-,-
230,2023-02-01 00:00:00,TXO,202302W2,13800.0,賣權,-,-,-,-,0.0,0.2,32.0,-,-,-,-,,一般,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7180,2023-02-08 00:00:00,TXO,202302W2,16800.0,賣權,-,-,-,-,0.0,0.0,0.0,-,-,-,-,,一般,-,-
7184,2023-02-08 00:00:00,TXO,202302W2,16900.0,賣權,-,-,-,-,0.0,0.0,0.0,-,-,-,-,,一般,-,-
7188,2023-02-08 00:00:00,TXO,202302W2,17000.0,賣權,-,-,-,-,0.0,0.0,1.0,-,-,-,-,,一般,-,-
7192,2023-02-08 00:00:00,TXO,202302W2,17100.0,賣權,-,-,-,-,0.0,0.0,0.0,-,-,-,-,,一般,-,-


In [29]:
# 取得 dataframe index
index = put_df['履約價'].unique()
idx = np.sort(index)
idx

array([13400.0, 13500.0, 13600.0, 13700.0, 13800.0, 13900.0, 14000.0,
       14100.0, 14200.0, 14300.0, 14400.0, 14500.0, 14550.0, 14600.0,
       14650.0, 14700.0, 14750.0, 14800.0, 14850.0, 14900.0, 14950.0,
       15000.0, 15050.0, 15100.0, 15150.0, 15200.0, 15250.0, 15300.0,
       15350.0, 15400.0, 15450.0, 15500.0, 15550.0, 15600.0, 15650.0,
       15700.0, 15750.0, 15800.0, 15850.0, 15900.0, 15950.0, 16000.0,
       16050.0, 16100.0, 16200.0, 16300.0, 16400.0, 16500.0, 16600.0,
       16700.0, 16800.0, 16900.0, 17000.0, 17100.0, 17200.0], dtype=object)

In [30]:
# 取得 dataframe 欄位名稱
dates = put_df['交易日期'].unique()
dates

array([datetime.datetime(2023, 2, 1, 0, 0),
       datetime.datetime(2023, 2, 2, 0, 0),
       datetime.datetime(2023, 2, 3, 0, 0),
       datetime.datetime(2023, 2, 6, 0, 0),
       datetime.datetime(2023, 2, 7, 0, 0),
       datetime.datetime(2023, 2, 8, 0, 0)], dtype=object)

In [31]:
# 建立 dataframe 包含 index 與欄位名稱
put_t_df = pd.DataFrame(index = idx, columns = list(dates))
# 取出各日期的未沖銷契約數 依序放入 dataframe 中
for col in put_t_df.columns:
    put_t_df[col] = put_df.loc[put_df['交易日期'] == col].set_index('履約價')['未沖銷契約數']
#空值填入 0
put_t_df = put_t_df.fillna(0)
put_t_df

,2023-02-01,2023-02-02,2023-02-03,2023-02-06,2023-02-07,2023-02-08
13400.0,140.0,140.0,140.0,140.0,140.0,140.0
13500.0,5.0,5.0,5.0,5.0,5.0,5.0
13600.0,0.0,0.0,0.0,0.0,0.0,0.0
13700.0,11.0,11.0,11.0,11.0,11.0,11.0
13800.0,32.0,32.0,32.0,32.0,32.0,32.0
13900.0,1.0,1.0,1.0,1.0,1.0,1.0
14000.0,80.0,80.0,80.0,80.0,80.0,80.0
14100.0,166.0,168.0,168.0,168.0,168.0,168.0
14200.0,465.0,461.0,461.0,479.0,479.0,481.0
14300.0,259.0,293.0,276.0,273.0,272.0,272.0


#Part 3: 數據分析

In [32]:
def get_options(option_df, contract_period, put_or_call, strike_price, trade_period):
    
    option_df = df.loc[(df['到期月份(週別)'] == contract_period) & \
                       (df['買賣權'] == put_or_call) & \
                       (df['履約價'] == strike_price) & \
                       (df['交易時段'] == trade_period)]
    return option_df

In [33]:
# 建立 dataframe 包含 index 與欄位名稱
call_cp_df = pd.DataFrame(index = idx, columns = list(dates))
# 取出各日期的收盤價 依序放入 dataframe 中
for col in call_cp_df.columns:
    call_cp_df[col] = call_df.loc[call_df['交易日期'] == col].set_index('履約價')['收盤價']
#空值填入 0
call_cp_df = call_cp_df.fillna(0)
call_cp_df

,2023-02-01,2023-02-02,2023-02-03,2023-02-06,2023-02-07,2023-02-08
13400.0,1890.0,-,-,-,-,-
13500.0,-,-,-,-,-,-
13600.0,-,-,-,-,-,-
13700.0,-,-,-,-,-,-
13800.0,-,-,-,-,-,-
13900.0,-,-,-,-,-,-
14000.0,-,-,-,-,-,-
14100.0,-,-,-,-,-,-
14200.0,-,-,-,-,-,-
14300.0,1030.0,-,-,-,-,-


In [46]:
# 建立 dataframe 包含 index 與欄位名稱
put_cp_df = pd.DataFrame(index = idx, columns = list(dates))
# 取出各日期的未沖銷契約數 依序放入 dataframe 中
for col in put_cp_df.columns:
    put_cp_df[col] = put_df.loc[put_df['交易日期'] == col].set_index('履約價')['收盤價']
#空值填入 0
put_cp_df = put_cp_df.fillna(0)
put_cp_df

,2023-02-01,2023-02-02,2023-02-03,2023-02-06,2023-02-07,2023-02-08
13400.0,-,-,-,-,-,-
13500.0,-,-,-,-,-,-
13600.0,-,-,-,-,-,-
13700.0,-,-,-,-,-,-
13800.0,-,-,-,-,-,-
13900.0,-,-,-,-,-,-
14000.0,0.8,-,-,-,-,-
14100.0,0.8,0.7,-,-,-,-
14200.0,1.0,0.4,0.3,0.2,-,0.5
14300.0,1.5,0.6,0.4,0.2,0.2,-


In [35]:
import plotly.graph_objects as go

# Read data from df
z_data = call_cp_df

fig = go.Figure(data=[go.Surface(z=z_data.values)])

fig.update_layout(title='call', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [47]:
z_data = put_cp_df

fig = go.Figure(data=[go.Surface(z=z_data.values)])

fig.update_layout(title='put', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [37]:
txo_15400_c = get_options(df, '202302W2', '買權', 15400, '一般')
txo_15500_c = get_options(df, '202302W2', '買權', 15500, '一般')
txo_15600_c = get_options(df, '202302W2', '買權', 15600, '一般')
txo_15700_c = get_options(df, '202302W2', '買權', 15700, '一般')

txo_15400_p = get_options(df, '202302W2', '賣權', 15400, '一般')
txo_15500_p = get_options(df, '202302W2', '賣權', 15500, '一般')
txo_15600_p = get_options(df, '202302W2', '賣權', 15600, '一般')
txo_15700_p = get_options(df, '202302W2', '賣權', 15700, '一般')

In [38]:
# Initialize figure with subplots
fig = make_subplots(
    rows=7, cols=2, subplot_titles=("買權", "賣權")
)

# Add traces
fig.add_trace(go.Scatter(x = txo_15400_c["交易日期"], y = txo_15400_c["收盤價"], name = "買權15400"), row=1, col=1)
fig.add_trace(go.Scatter(x = txo_15500_c["交易日期"], y = txo_15500_c["收盤價"], name = "買權15500"), row=2, col=1)
fig.add_trace(go.Scatter(x = txo_15600_c["交易日期"], y = txo_15600_c["收盤價"], name = "買權15600"), row=3, col=1)
fig.add_trace(go.Scatter(x = txo_15700_c["交易日期"], y = txo_15700_c["收盤價"], name = "買權15700"), row=4, col=1)

fig.add_trace(go.Scatter(x = txo_15400_p["交易日期"], y = txo_15400_p["收盤價"], name = "賣權15400"), row=1, col=2)
fig.add_trace(go.Scatter(x = txo_15500_p["交易日期"], y = txo_15500_p["收盤價"], name = "賣權15500"), row=2, col=2)
fig.add_trace(go.Scatter(x = txo_15600_p["交易日期"], y = txo_15600_p["收盤價"], name = "賣權15600"), row=3, col=2)
fig.add_trace(go.Scatter(x = txo_15700_p["交易日期"], y = txo_15700_p["收盤價"], name = "賣權15700"), row=4, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="15400", row=1, col=2)
fig.update_yaxes(title_text="15500", row=2, col=2)
fig.update_yaxes(title_text="15600", row=3, col=2)
fig.update_yaxes(title_text="15700", row=4, col=2)

# Update title and height
fig.update_layout(title_text="台指選 每日交易行情 - 202302W2", width = 1000, height=2000)

fig.show()

In [39]:
# 設定左右子圖
fig = make_subplots(
    rows = 1, 
    cols = 10, 
    horizontal_spacing = 0.02, 
    subplot_titles = ("2021-11-4 買權", "2021-11-4 賣權", "2021-11-5 買權", "2021-11-5 賣權", "2021-11-8 買權", "2021-11-8 賣權", "2021-11-9 買權", "2021-11-9 賣權", "2021-11-10 買權", "2021-11-10 賣權")
)

## 圖一
# 畫買權長條圖
fig.add_trace(go.Bar(y = call_t_df.index,
                     x = -call_t_df['2023-2-2'],
                     orientation = 'h',
                     name = '2023-2-2 Call',
                     text = ("(" + (call_t_df['2023-2-2'] - call_t_df['2023-2-1']).astype('int').astype('str') + ") " + call_t_df['2023-2-2'].astype('int').astype('str')),
                     marker = dict(color = call_df['未沖銷契約數'], colorscale = 'burg')), 
              row = 1, 
              col = 1)

# 畫賣權長條圖
fig.add_trace(go.Bar(y = put_t_df.index,
                     x = put_t_df['2023-2-2'],
                     orientation = 'h',
                     name = '2023-2-2 Put',
                     text = (put_t_df['2023-2-2'].astype('int').astype('str') + " (" + (put_t_df['2023-2-2'] - put_t_df['2023-2-1']).astype('int').astype('str') + ")"),
                     marker = dict(color = put_df['未沖銷契約數'], colorscale = 'darkmint')), 
              row = 1, 
              col = 2)


# 設定圖的x跟y軸標題
fig.update_xaxes(tickvals = [-15000, -12500, -10000, -7500, -5000, -2500, 0],
                 ticktext = ['15k', '12.5k', '10k', '7.5k', '5k', '2.5k', '0'], 
                 title_text = "未沖銷契約數",
                 row = 1, 
                 col = 1)

fig.update_xaxes(tickvals = [0, 2500, 5000, 7500, 10000, 12500, 15000],
                 ticktext = ['0', '2.5k', '5k', '7.5k', '10k', '12.5k', '15k'], 
                 title_text = "未沖銷契約數",
                 row = 1, 
                 col = 2)

fig.update_yaxes(autorange = "reversed", 
                 showticklabels = False, 
                 title_text = "履約價",
                 row = 1, 
                 col = 1)

fig.update_yaxes(autorange = "reversed", 
                 row = 1, 
                 col = 2)


## 圖二
# 畫買權長條圖
fig.add_trace(go.Bar(y = call_t_df.index,
                     x = -call_t_df['2023-2-3'],
                     orientation = 'h',
                     name = '2023-2-3 Call',
                     text = ("(" + (call_t_df['2023-2-3'] - call_t_df['2023-2-2']).astype('int').astype('str') + ") " + call_t_df['2023-2-3'].astype('int').astype('str')),
                     marker = dict(color = call_df['未沖銷契約數'], colorscale = 'burg')), 
              row = 1, 
              col = 3)

# 畫賣權長條圖
fig.add_trace(go.Bar(y = put_t_df.index,
                     x = put_t_df['2023-2-3'],
                     orientation = 'h',
                     name = '2023-2-3 Put',
                     text = (put_t_df['2023-2-3'].astype('int').astype('str') + " (" + (put_t_df['2023-2-3'] - put_t_df['2023-2-2']).astype('int').astype('str') + ")"),
                     marker = dict(color = put_df['未沖銷契約數'], colorscale = 'darkmint')), 
              row = 1, 
              col = 4)


# 設定圖的x跟y軸標題
fig.update_xaxes(tickvals = [-15000, -12500, -10000, -7500, -5000, -2500, 0],
                 ticktext = ['15k', '12.5k', '10k', '7.5k', '5k', '2.5k', '0'], 
                 title_text = "未沖銷契約數",
                 row = 1, 
                 col = 3)

fig.update_xaxes(tickvals = [0, 2500, 5000, 7500, 10000, 12500, 15000],
                 ticktext = ['0', '2.5k', '5k', '7.5k', '10k', '12.5k', '15k'], 
                 title_text = "未沖銷契約數",
                 row = 1, 
                 col = 4)

fig.update_yaxes(autorange = "reversed", 
                 showticklabels = False, 
                 row = 1, 
                 col = 3)

fig.update_yaxes(autorange = "reversed", 
                 row = 1, 
                 col = 4)


## 圖三
# 畫買權長條圖
fig.add_trace(go.Bar(y = call_t_df.index,
                     x = -call_t_df['2023-2-6'],
                     orientation = 'h',
                     name = '2023-2-6 Call',
                     text = ("(" + (call_t_df['2023-2-6'] - call_t_df['2023-2-3']).astype('int').astype('str') + ") " + call_t_df['2023-2-6'].astype('int').astype('str')),
                     marker = dict(color = call_df['未沖銷契約數'], colorscale = 'burg')), 
              row = 1, 
              col = 5)

# 畫賣權長條圖
fig.add_trace(go.Bar(y = put_t_df.index,
                     x = put_t_df['2023-2-6'],
                     orientation = 'h',
                     name = '2023-2-6 Put',
                     text = (put_t_df['2023-2-6'].astype('int').astype('str') + " (" + (put_t_df['2023-2-6'] - put_t_df['2023-2-3']).astype('int').astype('str') + ")"),
                     marker = dict(color = put_df['未沖銷契約數'], colorscale = 'darkmint')), 
              row = 1, 
              col = 6)


# 設定圖的x跟y軸標題
fig.update_xaxes(tickvals = [-15000, -12500, -10000, -7500, -5000, -2500, 0],
                 ticktext = ['15k', '12.5k', '10k', '7.5k', '5k', '2.5k', '0'], 
                 title_text = "未沖銷契約數",
                 row = 1, 
                 col = 5)

fig.update_xaxes(tickvals = [0, 2500, 5000, 7500, 10000, 12500, 15000],
                 ticktext = ['0', '2.5k', '5k', '7.5k', '10k', '12.5k', '15k'], 
                 title_text = "未沖銷契約數",
                 row = 1, 
                 col = 6)

fig.update_yaxes(autorange = "reversed", 
                 showticklabels = False, 
                 row = 1, 
                 col = 5)

fig.update_yaxes(autorange = "reversed", 
                 row = 1, 
                 col = 6)

## 圖四
# 畫買權長條圖
fig.add_trace(go.Bar(y = call_t_df.index,
                     x = -call_t_df['2023-2-7'],
                     orientation = 'h',
                     name = '2023-2-7 Call',
                     text = ("(" + (call_t_df['2023-2-7'] - call_t_df['2023-2-6']).astype('int').astype('str') + ") " + call_t_df['2023-2-7'].astype('int').astype('str')),
                     marker = dict(color = call_df['未沖銷契約數'], colorscale = 'burg')), 
              row = 1, 
              col = 7)

# 畫賣權長條圖
fig.add_trace(go.Bar(y = put_t_df.index,
                     x = put_t_df['2023-2-7'],
                     orientation = 'h',
                     name = '2023-2-7 Put',
                     text = (put_t_df['2023-2-7'].astype('int').astype('str') + " (" + (put_t_df['2023-2-7'] - put_t_df['2023-2-6']).astype('int').astype('str') + ")"),
                     marker = dict(color = put_df['未沖銷契約數'], colorscale = 'darkmint')), 
              row = 1, 
              col = 8)


# 設定圖的x跟y軸標題
fig.update_xaxes(tickvals = [-15000, -12500, -10000, -7500, -5000, -2500, 0],
                 ticktext = ['15k', '12.5k', '10k', '7.5k', '5k', '2.5k', '0'], 
                 title_text = "未沖銷契約數",
                 row = 1, 
                 col = 7)

fig.update_xaxes(tickvals = [0, 2500, 5000, 7500, 10000, 12500, 15000],
                 ticktext = ['0', '2.5k', '5k', '7.5k', '10k', '12.5k', '15k'], 
                 title_text = "未沖銷契約數",
                 row = 1, 
                 col = 8)

fig.update_yaxes(autorange = "reversed", 
                 showticklabels = False, 
                 row = 1, 
                 col = 7)

fig.update_yaxes(autorange = "reversed", 
                 row = 1, 
                 col = 8)

## 圖五
# 畫買權長條圖
fig.add_trace(go.Bar(y = call_t_df.index,
                     x = -call_t_df['2023-2-8'],
                     orientation = 'h',
                     name = '2023-2-8 Call',
                     text = ("(" + (call_t_df['2023-2-8'] - call_t_df['2023-2-7']).astype('int').astype('str') + ") " + call_t_df['2023-2-8'].astype('int').astype('str')),
                     marker = dict(color = call_df['未沖銷契約數'], colorscale = 'burg')), 
              row = 1, 
              col = 9)

# 畫賣權長條圖
fig.add_trace(go.Bar(y = put_t_df.index,
                     x = put_t_df['2023-2-8'],
                     orientation = 'h',
                     name = '2023-2-8 Put',
                     text = (put_t_df['2023-2-8'].astype('int').astype('str') + " (" + (put_t_df['2023-2-8'] - put_t_df['2023-2-7']).astype('int').astype('str') + ")"),
                     marker = dict(color = put_df['未沖銷契約數'], colorscale = 'darkmint')), 
              row = 1, 
              col = 10)


# 設定圖的x跟y軸標題
fig.update_xaxes(tickvals = [-15000, -12500, -10000, -7500, -5000, -2500, 0],
                 ticktext = ['15k', '12.5k', '10k', '7.5k', '5k', '2.5k', '0'], 
                 title_text = "未沖銷契約數",
                 row = 1, 
                 col = 9)

fig.update_xaxes(tickvals = [0, 2500, 5000, 7500, 10000, 12500, 15000],
                 ticktext = ['0', '2.5k', '5k', '7.5k', '10k', '12.5k', '15k'], 
                 title_text = "未沖銷契約數",
                 row = 1, 
                 col = 10)

fig.update_yaxes(autorange = "reversed", 
                 showticklabels = False, 
                 row = 1, 
                 col = 9)

fig.update_yaxes(autorange = "reversed", 
                 row = 1, 
                 col = 10)

# 設定圖的標題跟長寬
fig.update_layout(title_text = "臺指選擇權[202302W2] 未沖銷契約數 支撐壓力圖", 
                  width = 2400, 
                  height = 1300)

fig.show()

#Part 4: 估計波動率

In [40]:
class garchOneOne(object):
       
    def __init__(self, logReturns):
        self.logReturns = logReturns * 100
        self.sigma_2 = self.garch_filter(self.garch_optimization())
        self.coefficients = self.garch_optimization()
        
    def garch_filter(self, parameters):
        "Returns the variance expression of a GARCH(1,1) process."
        
        # Slicing the parameters list
        omega = parameters[0]
        alpha = parameters[1]
        beta = parameters[2]
        
        # Length of logReturns
        length = len(self.logReturns)
        
        # Initializing an empty array
        sigma_2 = np.zeros(length)
        
        # Filling the array, if i == 0 then uses the long term variance.
        for i in range(length):
            if i == 0:
                sigma_2[i] = omega / (1 - alpha - beta)
            else:
                sigma_2[i] = omega + alpha * self.logReturns[i-1]**2 + beta * sigma_2[i-1]
        
        return sigma_2 
        
    def garch_loglikehihood(self, parameters):
        "Defines the log likelihood sum to be optimized given the parameters."
        length = len(self.logReturns)
        
        sigma_2 = self.garch_filter(parameters)
        
        loglikelihood = - np.sum(-np.log(sigma_2) - self.logReturns**2 / sigma_2)
        return loglikelihood
    
    def garch_optimization(self):
        "Optimizes the log likelihood function and returns estimated coefficients"
        # Parameters initialization
        parameters = [.1, .05, .92]
        
        # Parameters optimization, scipy does not have a maximize function, so we minimize the opposite of the equation described earlier
        opt = scipy.optimize.minimize(self.garch_loglikehihood, parameters,
                                     bounds = ((.001,1),(.001,1),(.001,1)))
        
        variance = .01**2 * opt.x[0] / (1 - opt.x[1] - opt.x[2])   # Times .01**2 because it concerns squared returns
        
        return np.append(opt.x, variance)

In [41]:
r = requests.get("https://www.macromicro.me/charts/14588/taiwan-10-year-government-bond-yield") #將網頁資料GET下來
#print(r.text)
sp = BeautifulSoup(r.text,"html.parser") #將網頁資料以html.parser
sel = sp.select("div span.val") #取HTML標中的 <div class="title"></div> 中的<a>標籤存入sel
for s in sel:
    print(s.text)

In [42]:
S0 = 15400 # 初始股價
K = 15800 # 歐式買權履約價
T = 1/240 # 到期日所剩時間
r = 0.0117 # 無風險利率
sigma = 0.202 # 波動率

I = 1000000 # 模擬蒙地卡羅幾次
np.random.seed(1000) # 初始化偽隨機變數
z = np.random.standard_t(15,I) # 抽取標準常態分佈隨機亂數
#print(z)

ST = S0 * np.exp((r - sigma ** 2 / 2) *Ｔ + sigma * math.sqrt(T) * z) # 求算到期日股價
#print(ST)

hT = np.maximum(ST - K, 0) # 求算到期日選擇權內含價值
#print(hT)

C0 = math.exp(-r * T) * np.mean(hT) # 求算蒙地卡羅估計值
print('Value of the European call options = {:5.6f}.'.format(C0))

P0 = C0 + math.exp(-r * T) * K - S0
print('Value of the European put options = {:5.6f}.'.format(P0))

Ｎ = 0.5 * C0 / (S0 - math.exp(-r * T) * K) #累積分佈函數

d1 = ((math.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T)))
d2 = ((math.log(S0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T)))

value = (S0 * scipy.stats.norm.cdf(d1, 0.0, 1.0) - K * math.exp(-r * T) * scipy.stats.norm.cdf(d2, 0.0, 1.0))
print('{:5.6f}'.format(value))
vega = S0 * scipy.stats.norm.pdf(d1, 0.0, 1.0) * math.sqrt(T)
print('{:5.6f}'.format(vega))
#IV = 0.5 * (27.83 + 27.44)
IV = 0.00
for i in range(10):
    IV -= (value - C0) / vega
    
print('IV of the European call options = {:3.4%}.'.format(IV))
 #隱含波動率

#%run bsm_mcs_euro.py

Value of the European call options = 3.600441.
Value of the European put options = 402.830210.
1.906537
58.513140
IV of the European call options = 28.9491%.


In [43]:
class BlackScholesModel(object):
    pass
    def __init__(self):
        pass

In [44]:
def option_info(id='TXO',sd='2023/01/01',ed='2023/01/31'): #搜集選擇權資訊  
    #下載選擇權資料
    opt_http = urllib3.PoolManager()
    opt_url = "https://www.taifex.com.tw/cht/3/dlOptDataDown"
    opt_res = otp_http.request('POST',url,
                               fields={
                                   'down_type': 1,
                                   'commodity_id': id,
                                   'query_start_date': sd,
                                   'query_end_date': ed
                                   }
                               )
    opt_html = opt_res.data
    #使用BeautifulSoup解析資料
    opt_soup = BeautifulSoup(opt_html, 'html.parser')
    opt_str = str(opt_soup)
    opt_line = opt_str.split('\r\n')
    #新增空dataframe並定義欄名
    opt_data = pd.DataFrame(columns = opt_line[0].split(','))
    #將資料一行一行填入dataframe
    for i in range(1, len(opt_line) - 1):
        opt_row_data = opt_line[i].split(',')[:-1]
        opt_data_len = len(opt_data)
        opt_data.loc[opt_data_len] = opt_row_data

In [45]:
!pip install arch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.3/907.3 KB 12.9 MB/s eta 0:00:00
